In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from preprocessing import process_text
from nltk import flatten

In [69]:
df = pd.read_csv('spam.csv', encoding="ISO-8859-1")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.rename(columns={'v1': 'Label', 'v2': 'Mail'}, inplace=True)
print(df['Label'].value_counts())

ham     4825
spam     747
Name: Label, dtype: int64


In [70]:
df['Label'] = df['Label'].map({'ham': 1, 'spam': 0})
df['Msg'] = df['Mail'].apply(lambda x: process_text(x))
df.head()

,Label,Mail,Msg
0,1,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,1,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,0,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,1,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,1,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, go, usf, life, around, though]"


In [71]:
all_words =list(flatten((list(df['Msg']))))
print("Length all words: {}".format(len(all_words)))
unique_words=list(set(all_words))
print("Length of unique words: {}".format(len(unique_words)))

Length all words: 49999
Length of unique words: 7937


In [72]:
ham_df = df[df["Label"] == 1]
spam_df = df[df["Label"] ==0]
ham_df.head()


,Label,Mail,Msg
0,1,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,1,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
3,1,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,1,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, go, usf, life, around, though]"
6,1,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, treat, like, aid,..."


In [73]:
spam_df.head()

,Label,Mail,Msg
2,0,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, wkly, comp, win, fa, cup, final,..."
5,0,FreeMsg Hey there darling it's been 3 week's n...,"[freemsg, hey, darling, week, word, back, id, ..."
8,0,WINNER!! As a valued network customer you have...,"[winner, valued, network, customer, selected, ..."
9,0,Had your mobile 11 months or more? U R entitle...,"[mobile, month, u, r, entitled, update, latest..."
11,0,"SIX chances to win CASH! From 100 to 20,000 po...","[six, chance, win, cash, pound, txt, csh, send..."


In [80]:
positive_words = list(flatten((list(ham_df['Msg']))))
negative_words = list(flatten((list(spam_df['Msg']))))

In [81]:
word_map={}
for word in unique_words:
    if unique_words.count(word)>1:
        pass
        # print('True')
    # word_map[word]={'pos':positive_words.count(word),'neg':negative_words.count(word)}


In [82]:
print(len(positive_words),len(negative_words),len(all_words))

39090 10909 49999


In [83]:
negative_words[:10]

['free', 'entry', 'wkly', 'comp', 'win', 'fa', 'cup', 'final', 'tkts', 'st']

In [79]:
positive_words[:10]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la']